In [ ]:
# The code was removed by Watson Studio for sharing.

In [1]:
import pandas as pd
import numpy as np
import types
import pandas as pd
from botocore.client import Config

In [2]:
# The code was removed by Watson Studio for sharing.

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


Removing the redundant column and column which has no meaning full data 

In [3]:
df.replace(r'^\s*$', np.nan, regex=True)
df.drop(['OBJECTID','INCKEY','COLDETKEY','REPORTNO','STATUS','INTKEY','EXCEPTRSNCODE','EXCEPTRSNDESC','SEVERITYCODE.1'],axis = 1,inplace=True)
df.drop(['CROSSWALKKEY','SEGLANEKEY','ST_COLCODE','SPEEDING','SDOTCOLNUM','PEDROWNOTGRNT','INATTENTIONIND','SDOT_COLCODE','JUNCTIONTYPE'],axis = 1,inplace=True)

In [4]:
df.drop(['INCDATE'],axis = 1,inplace=True)

In [5]:
df.head()

,SEVERITYCODE,X,Y,ADDRTYPE,LOCATION,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDTTM,SDOT_COLDESC,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,ST_COLDESC,HITPARKEDCAR
0,2,-122.323148,47.703140,Intersection,5TH AVE NE AND NE 103RD ST,Injury Collision,Angles,2,0,0,2,3/27/2013 2:54:00 PM,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",N,Overcast,Wet,Daylight,Entering at angle,N
1,1,-122.347294,47.647172,Block,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N,Property Damage Only Collision,Sideswipe,2,0,0,2,12/20/2006 6:55:00 PM,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",0,Raining,Wet,Dark - Street Lights On,From same direction - both going straight - bo...,N
2,1,-122.334540,47.607871,Block,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST,Property Damage Only Collision,Parked Car,4,0,0,3,11/18/2004 10:20:00 AM,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",0,Overcast,Dry,Daylight,One parked--one moving,N
3,1,-122.334803,47.604803,Block,2ND AVE BETWEEN MARION ST AND MADISON ST,Property Damage Only Collision,Other,3,0,0,3,3/29/2013 9:26:00 AM,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",N,Clear,Dry,Daylight,From same direction - all others,N
4,2,-122.306426,47.545739,Intersection,SWIFT AVE S AND SWIFT AV OFF RP,Injury Collision,Angles,2,0,0,2,1/28/2004 8:04:00 AM,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",0,Raining,Wet,Daylight,Entering at angle,N


In [7]:
df.drop(df[df['ROADCOND'].isnull()].index,axis = 0,inplace=True)
df.drop(df[df['LIGHTCOND'].isnull()].index,axis = 0,inplace=True)
df.drop(df[df['WEATHER'].isnull()].index,axis = 0,inplace=True)
df.drop(df[df['ADDRTYPE'].isnull()].index,axis = 0,inplace=True)

In [6]:
!pip install geopandas
!pip install geopy
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

     |████████████████████████████████| 972kB 9.3MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 45.0MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 11.5MB/s eta 0:00:01
     |████████████████████████████████| 14.8MB 21.4MB/s eta 0:00:01


converting the X and Y co-ordinates to point from which we can find the location of city

In [19]:
df['Location1'] = ""
locator = Nominatim(user_agent="myGeocoder")
df['coordinates'] = ""
geolocator = Nominatim(user_agent="geoapiExercises")
df['coordinates'] = df[['Y','X']].apply(lambda x : '{},{}'.format(x[0],x[1]), axis=1)

In [20]:
df.head()

,SEVERITYCODE,X,Y,ADDRTYPE,LOCATION,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,...,INCDTTM,SDOT_COLDESC,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,ST_COLDESC,HITPARKEDCAR,Location1,coordinates
0,2,-122.323148,47.703140,Intersection,5TH AVE NE AND NE 103RD ST,Injury Collision,Angles,2,0,0,...,3/27/2013 2:54:00 PM,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",N,Overcast,Wet,Daylight,Entering at angle,N,,"47.70314032,-122.32314840000002"
1,1,-122.347294,47.647172,Block,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N,Property Damage Only Collision,Sideswipe,2,0,0,...,12/20/2006 6:55:00 PM,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",0,Raining,Wet,Dark - Street Lights On,From same direction - both going straight - bo...,N,,"47.64717249,-122.3472943"
2,1,-122.334540,47.607871,Block,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST,Property Damage Only Collision,Parked Car,4,0,0,...,11/18/2004 10:20:00 AM,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",0,Overcast,Dry,Daylight,One parked--one moving,N,,"47.60787143,-122.33453999999999"
3,1,-122.334803,47.604803,Block,2ND AVE BETWEEN MARION ST AND MADISON ST,Property Damage Only Collision,Other,3,0,0,...,3/29/2013 9:26:00 AM,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",N,Clear,Dry,Daylight,From same direction - all others,N,,"47.60480289,-122.3348033"
4,2,-122.306426,47.545739,Intersection,SWIFT AVE S AND SWIFT AV OFF RP,Injury Collision,Angles,2,0,0,...,1/28/2004 8:04:00 AM,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",0,Raining,Wet,Daylight,Entering at angle,N,,"47.54573946,-122.3064263"


In [24]:
for a in zip(df['coordinates']):
    location = geolocator.reverse(a,exactly_one=True)
    address = location.raw['address'] 
    df['Location1'] = address.get('city', '')

ValueError: Must be a coordinate pair or Point

In [25]:
df['county'] = ""
for a in zip(df['coordinates']):
    location = geolocator.reverse(a,exactly_one=True)
    address = location.raw['address'] 
    df['county'] = address.get('county', '')

ValueError: Must be a coordinate pair or Point

In [26]:
cod=47.66716211,-122.3762169
location = geolocator.reverse(cod,exactly_one=True)
address = location.raw['address']
city = address.get('city', '')
print(city)
print(location.raw)

Seattle
{'place_id': 124400942, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 165758398, 'lat': '47.6669773', 'lon': '-122.37584739878048', 'display_name': "O'Reilly Auto Parts, 5220, 15th Avenue Northwest, Ballard, Seattle, King County, Washington, 98107, United States of America", 'address': {'shop': "O'Reilly Auto Parts", 'house_number': '5220', 'road': '15th Avenue Northwest', 'suburb': 'Ballard', 'city': 'Seattle', 'county': 'King County', 'state': 'Washington', 'postcode': '98107', 'country': 'United States of America', 'country_code': 'us'}, 'boundingbox': ['47.6668792', '47.667076', '-122.3760399', '-122.3756545']}


In [27]:
df['county'] = "King County"
df['Location1'] ="Seattle"

In [28]:
df.head()

,SEVERITYCODE,X,Y,ADDRTYPE,LOCATION,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,...,SDOT_COLDESC,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,ST_COLDESC,HITPARKEDCAR,Location1,coordinates,county
0,2,-122.323148,47.703140,Intersection,5TH AVE NE AND NE 103RD ST,Injury Collision,Angles,2,0,0,...,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",N,Overcast,Wet,Daylight,Entering at angle,N,Seattle,"47.70314032,-122.32314840000002",King County
1,1,-122.347294,47.647172,Block,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N,Property Damage Only Collision,Sideswipe,2,0,0,...,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",0,Raining,Wet,Dark - Street Lights On,From same direction - both going straight - bo...,N,Seattle,"47.64717249,-122.3472943",King County
2,1,-122.334540,47.607871,Block,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST,Property Damage Only Collision,Parked Car,4,0,0,...,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",0,Overcast,Dry,Daylight,One parked--one moving,N,Seattle,"47.60787143,-122.33453999999999",King County
3,1,-122.334803,47.604803,Block,2ND AVE BETWEEN MARION ST AND MADISON ST,Property Damage Only Collision,Other,3,0,0,...,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",N,Clear,Dry,Daylight,From same direction - all others,N,Seattle,"47.60480289,-122.3348033",King County
4,2,-122.306426,47.545739,Intersection,SWIFT AVE S AND SWIFT AV OFF RP,Injury Collision,Angles,2,0,0,...,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",0,Raining,Wet,Daylight,Entering at angle,N,Seattle,"47.54573946,-122.3064263",King County


In [44]:
df['INCDTTM'] = pd.to_datetime(df['INCDTTM'])
df['dayofweek'] = df['INCDTTM'].dt.dayofweek

In [45]:
df['dayofweek'] = np.where((df.dayofweek == '0.0'),'Monday',df.dayofweek)

In [46]:
df['dayofweek'] = np.where((df.dayofweek == '1.0'),'Tuesday',df.dayofweek)
df['dayofweek'] = np.where((df.dayofweek == '2.0'),'Wednesday',df.dayofweek)
df['dayofweek'] = np.where((df.dayofweek == '3.0'),'Thursday',df.dayofweek)
df['dayofweek'] = np.where((df.dayofweek == '4.0'),'Friday',df.dayofweek)
df['dayofweek'] = np.where((df.dayofweek == '5.0'),'Saturday',df.dayofweek)
df['dayofweek'] = np.where((df.dayofweek == '6.0'),'Sunday',df.dayofweek)

In [47]:
df.drop(df[df['ROADCOND'].isnull()].index,axis = 0,inplace=True)
df.drop(df[df['LIGHTCOND'].isnull()].index,axis = 0,inplace=True)
df.drop(df[df['WEATHER'].isnull()].index,axis = 0,inplace=True)
df.drop(df[df['UNDERINFL'].isnull()].index,axis = 0,inplace=True)
df.drop(df[df['dayofweek'].isnull()].index,axis = 0,inplace=True)
df.drop(df[df['SEVERITYCODE'].isnull()].index,axis = 0,inplace=True)

In [48]:
df['UNDERINFL'] = np.where((df.UNDERINFL == 'N'),'0',df.UNDERINFL)
df['UNDERINFL'] = np.where((df.UNDERINFL == 'Y'),'1',df.UNDERINFL)

In [49]:
df.drop(df[df['WEATHER']=='Unknown'].index,axis = 0, inplace = True)
df.drop(df[df['WEATHER']=='Other'].index,axis = 0, inplace = True)

In [50]:
df.drop(df[df['ROADCOND']=='Unknown'].index,axis = 0, inplace = True)
df.drop(df[df['ROADCOND']=='Other'].index,axis = 0, inplace = True)

In [51]:
df.drop(df[df['LIGHTCOND']=='Dark - Unknown Lighting'].index,axis = 0, inplace = True)

In [52]:
df.drop(df[df['LIGHTCOND']=='Unknown'].index,axis = 0, inplace = True)
df.drop(df[df['LIGHTCOND']=='Other'].index,axis = 0, inplace = True)

In [53]:
# The code was removed by Watson Studio for sharing.

{'file_name': 'Transform-Data.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'corseracapstone-donotdelete-pr-xntjynexnacup6',
 'asset_id': '453bef29-120c-47ac-be34-e1c9b8368820'}

In [54]:
df['INCDTTM'] = pd.to_datetime(df['INCDTTM'])
df['dayofweek'] = df['INCDTTM'].dt.dayofweek

In [55]:
df.head()

,SEVERITYCODE,X,Y,ADDRTYPE,LOCATION,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,...,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,ST_COLDESC,HITPARKEDCAR,Location1,coordinates,county,dayofweek
0,2,-122.323148,47.703140,Intersection,5TH AVE NE AND NE 103RD ST,Injury Collision,Angles,2,0,0,...,0,Overcast,1,Daylight,Entering at angle,N,Seattle,"47.70314032,-122.32314840000002",King County,2
1,1,-122.347294,47.647172,Block,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N,Property Damage Only Collision,Sideswipe,2,0,0,...,0,Raining,1,Dark - Street Lights On,From same direction - both going straight - bo...,N,Seattle,"47.64717249,-122.3472943",King County,2
2,1,-122.334540,47.607871,Block,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST,Property Damage Only Collision,Parked Car,4,0,0,...,0,Overcast,0,Daylight,One parked--one moving,N,Seattle,"47.60787143,-122.33453999999999",King County,3
3,1,-122.334803,47.604803,Block,2ND AVE BETWEEN MARION ST AND MADISON ST,Property Damage Only Collision,Other,3,0,0,...,0,Clear,0,Daylight,From same direction - all others,N,Seattle,"47.60480289,-122.3348033",King County,4
4,2,-122.306426,47.545739,Intersection,SWIFT AVE S AND SWIFT AV OFF RP,Injury Collision,Angles,2,0,0,...,0,Raining,1,Daylight,Entering at angle,N,Seattle,"47.54573946,-122.3064263",King County,2


In [56]:
df['ROADCOND'] = np.where((df.ROADCOND == 'Dry'),0,df.ROADCOND)
df['ROADCOND'] = np.where((df.ROADCOND == 'Wet'),1,df.ROADCOND)
df['ROADCOND'] = np.where((df.ROADCOND == 'Ice'),2,df.ROADCOND)
df['ROADCOND'] = np.where((df.ROADCOND == 'Snow/Slush'),3,df.ROADCOND)
df['ROADCOND'] = np.where((df.ROADCOND == 'Standing Water'),4,df.ROADCOND)
df['ROADCOND'] = np.where((df.ROADCOND == 'Sand/Mud/Dirt'),5,df.ROADCOND)
df['ROADCOND'] = np.where((df.ROADCOND == 'Oil'),6,df.ROADCOND)

In [57]:
df['LIGHTCOND'] = np.where((df.LIGHTCOND == 'Daylight'),0,df.LIGHTCOND)
df['LIGHTCOND'] = np.where((df.LIGHTCOND == 'Dark - Street Lights On'),1,df.LIGHTCOND)
df['LIGHTCOND'] = np.where((df.LIGHTCOND == 'Dusk'),2,df.LIGHTCOND)
df['LIGHTCOND'] = np.where((df.LIGHTCOND == 'Dawn'),3,df.LIGHTCOND)
df['LIGHTCOND'] = np.where((df.LIGHTCOND == 'Dark - No Street Lights'),4,df.LIGHTCOND)
df['LIGHTCOND'] = np.where((df.LIGHTCOND == 'Dark - Street Lights Off'),4,df.LIGHTCOND)

In [58]:
df['WEATHER'] = np.where((df.WEATHER == 'Clear'),0,df.WEATHER)
df['WEATHER'] = np.where((df.WEATHER == 'Raining'),1,df.WEATHER)
df['WEATHER'] = np.where((df.WEATHER == 'Overcast'),2,df.WEATHER)
df['WEATHER'] = np.where((df.WEATHER == 'Snowing'),3,df.WEATHER)
df['WEATHER'] = np.where((df.WEATHER == 'Fog/Smog/Smoke'),4,df.WEATHER)
df['WEATHER'] = np.where((df.WEATHER == 'Sleet/Hail/Freezing Rain'),5,df.WEATHER)
df['WEATHER'] = np.where((df.WEATHER == 'Blowing Sand/Dirt'),6,df.WEATHER)
df['WEATHER'] = np.where((df.WEATHER == 'Severe Crosswind'),7,df.WEATHER)
df['WEATHER'] = np.where((df.WEATHER == 'Partly Cloudy'),8,df.WEATHER)

In [ ]:
project.save_data("Transform-Data-1.csv", data=df.to_csv(index=False),overwrite=True)